In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error

In [2]:
import my_models as my_models

In [3]:
net3_ped1 = my_models.Net3_ped1()
net3_ped2 = my_models.Net3_ped2()
net3_ped3 = my_models.Net3_ped3()
net3_ped4 = my_models.Net3_ped4()
net3_ped5 = my_models.Net3_ped5()
net3_ped6 = my_models.Net3_ped6()
net3_ped7 = my_models.Net3_ped7()
net3_ped8 = my_models.Net3_ped8()
net3_ped9 = my_models.Net3_ped9()

In [5]:
net3_ped_repulse_total = my_models.Net3_ped_repulse_total(net3_ped1, net3_ped2, net3_ped3, net3_ped4, net3_ped5,
                                                net3_ped6, net3_ped7, net3_ped8, net3_ped9)
net3_attract = my_models.Net3_attract()
net3_repulse = my_models.Net3_repulse()
net3_ped_group = my_models.Net3_ped_group()

In [7]:
path_ped_repulse = "weights/Net3_ped_repulse_total.pth"
path_attract = "weights/net3_attract.pth"
path_repulse = "weights/net3_repulse.pth"
path_group = "weights/net3_ped_group.pth"

In [8]:
net3_ped_repulse_total.load_state_dict(torch.load(path_ped_repulse))
net3_attract.load_state_dict(torch.load(path_attract))
net3_repulse.load_state_dict(torch.load(path_repulse))
net3_ped_group.load_state_dict(torch.load(path_group))

<All keys matched successfully>

In [9]:
net3_complete = my_models.Net3_complete(net3_attract, net3_repulse, net3_ped_repulse_total, net3_ped_group)
# print(net3_complete)

In [10]:
path_net3_complete = "weights/net3_complete2.pth"

In [11]:
net3_complete.load_state_dict(torch.load(path_net3_complete))

<All keys matched successfully>

In [12]:
from scipy.interpolate import interp1d

def input_interpolated(temp_xy):
    # takes the 3 timesteps of inputs and rescales it into 10 timesteps
    # returns as (10, 2)
    temp_x = np.array((temp_xy[0], temp_xy[2], temp_xy[4]))
    temp_y = np.array((temp_xy[1], temp_xy[3], temp_xy[5]))
    x_total = []
    y_total = []
    for p in range(2):
        x = temp_x[0+p:2+p]
        y = temp_y[0+p:2+p]
        if (x[0] == x[-1] and y[0] == y[-1]):
            # if both x values are same, we do this to avoid nan given by interp1d
            x_new = np.linspace(x[0], x[-1], num=5)
            y_new = np.linspace(y[0], y[-1], num=5)
        else:            
            f2 = interp1d(x, y)
            x_new = np.linspace(x[0], x[-1], num=5)
            
            y_new = f2(x_new)
        x_total.append(x_new)
        y_total.append(y_new)
    x_total = np.array(x_total)
    x_total = x_total.flatten()
    y_total = np.array(y_total)
    y_total = y_total.flatten()
    xy_input = []
    for p in range(len(x_total)):
        temp_array = [x_total[p], y_total[p]]
        xy_input.append(temp_array)
    xy_input = np.array(xy_input)
    return xy_input

In [14]:
test_df_path = "data/test_data/eth_data_prepared_normal3.csv"
# delta_t = 0.4

In [15]:
test_df = pd.read_csv(test_df_path, index_col=None)

In [16]:
test_df.head()

frame_id  ped_no      pos_x     pos_y     vel_x     vel_y  rotation_angle  \
0       780       1   8.456844  3.588066  1.671714  0.176292             0.0   
1       786       1   9.125530  3.658583  1.662877  0.326723             0.0   
2       792       1   9.787146  3.849445  1.683334  0.371084             0.0   
3       798       1  10.472197  3.955450  1.598568  0.264795             0.0   
4       804       1  11.066000  4.061280  1.574526  0.456390             0.0   

   eta_group_x  eta_group_y  bound_x  ...  n_w_y21  n_w_y22  n_w_y23  n_w_y24  \
0          0.0          0.0    0.793  ...      0.0      0.0      0.0      0.0   
1          0.0          0.0    0.793  ...      0.0      0.0      0.0      0.0   
2          0.0          0.0    0.793  ...      0.0      0.0      0.0      0.0   
3          0.0          0.0    0.793  ...      0.0      0.0      0.0      0.0   
4          0.0          0.0    0.793  ...      0.0      0.0      0.0      0.0   

   n_w_y25  n_w_y26  n26_y  d_w_bound  n_w_bound_x  n_w_bound_y  
0      0.0      0.0    0.0   8.906774     0.949485     0.313814  
1      0.0      0.0    0.0   9.564877     0.954067     0.299594  
2      0.0      0.0    0.0  10.253295     0.954537     0.298094  
3      0.0      0.0    0.0  10.939286     0.957302     0.289091  
4      0.0      0.0    0.0  11.538544     0.959047     0.283249  

[5 rows x 144 columns]

In [19]:
from collections import Counter

In [20]:
tempppp = Counter(np.array(test_df['ped_no']))
# tempppp
# 171

In [29]:
def speed_finder(positions):
    # temp_seq = np.reshape(positions, (10, 2))
    positions = positions.view((10, 2))
    # print(positions)
    temp_array = []
    # global goal
    for k in reversed(range(10)):
        # print(k)
        temp_positions = positions[k] - positions[k - 1]
        temp_positions = torch.norm(temp_positions)
        # print(temp_positions)
        # temp_positions = temp_positions.tolist()
        # print(temp_positions)
        temp_array.insert(0, temp_positions)
    return torch.Tensor([temp_array])

In [30]:
import math

In [31]:
# make a class prediction for one row of data
# give previous velocity directly here
#  e, x_attract1, x_attract2, x_bound, x1, x2, x3, x4, x5, x6, x7, x8, x9, x_group, x_speed
def predict_synthetic(model, delta_t, last_velocity, temp_input, temp_goal, temp_n1, temp_n2, temp_n3, temp_n4, temp_n5,
                      temp_n6, temp_n7, temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid):

    previous_velocity = last_velocity
    previous_speed = np.linalg.norm(previous_velocity)
    temp_input2 = temp_input - temp_input[:, 0]
    temp_input_1 = speed_finder(temp_input2)/delta_t
    x_speed = temp_input_1[:, -1:]
    # make prediction
    # yhat = net3_complete(goal2, x_attract1, x_attract2, x_bound, x1, x2, x3, x4, x5, x6, x7, x8, x9, x_group, x_speed)
    yhat = model(temp_goal, temp_input_1, temp_input2, temp_n_bound, temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, 
                 temp_n6, temp_n7, temp_n8, temp_n9, temp_n_group, temp_n_centroid)
    
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    # that is actually not the desired velocity: try giving it directly

    desired_velocity = previous_velocity + (yhat * delta_t)
    desired_speed = np.linalg.norm(desired_velocity)
    
    # this section is to cap the velocity to a certain value
    factor = np.minimum(1.0, (1.3 * previous_speed) / desired_speed)
    # if factor != 1.0 : print("different factor")
    if desired_speed == 0:
        factor = 0.0
    # print(factor)
    current_velocity = desired_velocity * factor
    # print("current velocity: ", current_velocity, "\n")
    # print("current position: ", temp_input[0, -2:].detach().numpy(), "\n")
    next_position = temp_input[0, -2:].detach().numpy() + (current_velocity * delta_t)
     
    return yhat, next_position

In [32]:
ped_array = np.unique(np.array(test_df['ped_no']))
for j in range(3):
    # if j == 5: break
    test_df_indiv2 = test_df.loc[test_df['ped_no'] == ped_array[j]]
    # print(test_df_indiv2.head())

In [33]:
test_df_indiv = test_df.loc[test_df['ped_no'] == 359]
# test_df_indiv = test_df_indiv.loc[test_df_indiv['frame_id'] >= 1242]

In [34]:
def nearest_neighbour_finder(temp_neighbours):
    a = temp_neighbours[0]
    b = np.argsort(a[np.nonzero(a)])[:9]
    b = b + 1
    label_dw = "d_w"
    label_nw_x = "n_w_x"
    label_nw_y = "n_w_y"
    label_dw_list = []
    label_nw_x_list = []
    label_nw_y_list = []
    for r in range(len(b)):
        label_dw2 = label_dw + str(b[r])
        label_nw_x2 = label_nw_x + str(b[r])
        label_nw_y2 = label_nw_y + str(b[r])
        label_dw_list.append(label_dw2)
        label_nw_x_list.append(label_nw_x2)
        label_nw_y_list.append(label_nw_y2)

    label_dw_list = np.array(label_dw_list)
    label_nw_x_list = np.array(label_nw_x_list)
    label_nw_y_list = np.array(label_nw_y_list)
    return label_dw_list, label_nw_x_list, label_nw_y_list

In [ ]:
dest_list = np.array(((-2, 5.8566027), (-6.5902743, 0.065724367), (-6.5553084, 1.1867515), (15.10717, 5.5659299)))
dest_list

In [38]:
def goal_estimator2(given_dest, input_xy):
    goal_estimated = (given_dest - input_xy[0])/np.linalg.norm((given_dest - input_xy[0]))
    goal_estimated = torch.Tensor([goal_estimated])
    return goal_estimated

In [39]:
def get_last_disp(y_true, y_prediction):
    last_disp = np.linalg.norm((y_true[-1, :] - y_prediction[-1, :]))
    return last_disp

def get_eucledian(true_values, predicted_values):
    eucledian_v = np.linalg.norm((true_values - predicted_values), axis =1)
    eucledian_v = np.mean(eucledian_v)
    return eucledian_v

In [40]:
dest_df = pd.read_csv("filter/eth_estimated_dests2.csv", index_col=0)

In [41]:
dest_df.head()

dest_x     dest_y
1  -6.555308  11.867515
2  -2.000000   5.856603
3  -2.000000   5.856603
4  15.107170   5.565930
5  15.107170   5.565930

In [42]:
estimated_dest_list = np.array(dest_df[["dest_x", "dest_y"]])

In [43]:
estimated_dest_list.shape

(360, 2)

In [44]:
# to predict all the trajectories, 0.1 seconds of prediction at a time
# input of one seconds
# next step: make it into a function
# function to get predictions of trajectories, recursively
def get_predictions(step_size, delta_t, temp_cycle, input_xy, velocity, n1, n2, n3, n4, n5, n6, n7, n8, n9, n_bound,
                    n_group, n_centroid, estimated_dest):
    dest_list = np.array(((-2, 5.8566027), (-6.5902743, 0.065724367), (-6.5553084, 11.867515), (15.10717, 5.5659299)))
    step_size = step_size
    delta_t = delta_t
    temp_cycle = temp_cycle
    predicted_trajs = []
    real_trajs = []
    force_pred = []
    interpolate = True
    my_goal = goal_estimator2(estimated_dest, input_xy)
    # len(input_xy)
    end_cycle = len(input_xy)
    for p in range(0, end_cycle, step_size):
        # print("timesteps: ", p)
        start_cycle = temp_cycle + p
        if (start_cycle + step_size) > end_cycle: 
            # print("i am here")
            continue

        if (temp_cycle != 10):
            if not interpolate:
                temp_input5 = np.concatenate((np.zeros(14), input_xy[(start_cycle - temp_cycle):start_cycle].flatten()))
                temp_input = torch.Tensor([temp_input5])
            else:
                temp_input_total = input_xy[(start_cycle - temp_cycle):start_cycle].flatten()
                temp_input5 = input_interpolated(temp_input_total).flatten()
                temp_input = torch.Tensor([temp_input5])
        else: 
            temp_input = torch.Tensor([input_xy[(start_cycle - temp_cycle):start_cycle].flatten()])

        temp_real = input_xy[(start_cycle):(start_cycle + step_size)]
        # print((start_cycle - 10), start_cycle)
        # print(start_cycle, (start_cycle + step_size))
        real_trajs.append(temp_real)
        prediction_list = []
        next_position_list = []
        # print("input: ", temp_input, "\n")
        for x in range(step_size):
            if x == 0:
                temp_input3 = temp_input
            else: 
                if not interpolate:
                    temp_input3 = torch.cat((temp_input[:, 2:], torch.Tensor(next_position)), axis=1)
                    temp_input = temp_input3
                else:
                    temp_input_total = np.concatenate((temp_input_total[2:], next_position.flatten()))
                    temp_input3 = input_interpolated(temp_input_total).flatten()
                    temp_input3 = torch.Tensor([temp_input3])
                    temp_input = temp_input3

            temp_velocity = velocity[(start_cycle -1 + x)]
            temp_goal = my_goal
            temp_n1 = torch.Tensor([n1[start_cycle + x].flatten()])
            temp_n2 = torch.Tensor([n2[start_cycle + x].flatten()])
            temp_n3 = torch.Tensor([n3[start_cycle + x].flatten()])
            temp_n4 = torch.Tensor([n4[start_cycle + x].flatten()])
            temp_n5 = torch.Tensor([n5[start_cycle + x].flatten()])
            temp_n6 = torch.Tensor([n6[start_cycle + x].flatten()])
            temp_n7 = torch.Tensor([n7[start_cycle + x].flatten()])
            temp_n8 = torch.Tensor([n8[start_cycle + x].flatten()])
            temp_n9 = torch.Tensor([n9[start_cycle + x].flatten()])
            temp_n_bound = torch.Tensor([n_bound[start_cycle + x].flatten()])
            temp_n_group = torch.Tensor([n_group[start_cycle + x].flatten()])
            temp_n_centroid = torch.Tensor([n_centroid[start_cycle + x].flatten()])

            prediction1, next_position = predict_synthetic(net3_complete, delta_t, temp_velocity,  temp_input3, temp_goal, 
                                                           temp_n1, temp_n2, temp_n3, temp_n4, temp_n5, temp_n6, temp_n7, 
                                                           temp_n8, temp_n9, temp_n_bound, temp_n_group, temp_n_centroid)
            prediction_list.append(prediction1)
            next_position_list.append(next_position)


        prediction_list = np.array(prediction_list)
        next_position_list = np.array(next_position_list)
        predicted_trajs.append(next_position_list)
        force_pred.append(prediction_list)

    predicted_trajs = np.array(predicted_trajs)
    force_pred = np.array(force_pred)
    real_trajs = np.array(real_trajs)

    predicted_trajs_reshaped = np.reshape(predicted_trajs, (int((len(predicted_trajs.flatten())/2)), 2))
    real_trajs_reshaped = np.reshape(real_trajs, (int((len(real_trajs.flatten())/2)), 2))
    # print(real_trajs.shape)
    # print(predicted_trajs.shape)

    eucledian_temp_list = []
    for k in range(0, len(predicted_trajs_reshaped), step_size):
        eucledian_temp = get_eucledian(real_trajs_reshaped[0+k:12+k], predicted_trajs_reshaped[0+k:12+k])
        # print(eucledian_temp)
        eucledian_temp_list.append(eucledian_temp)
    eucledian_temp_list = np.array(eucledian_temp_list)
    final_eucledian_dist = np.mean(eucledian_temp_list)
    final_disp = get_last_disp(real_trajs_reshaped, predicted_trajs_reshaped)

    return final_eucledian_dist, final_disp

In [45]:
ped_array = np.unique(np.array(test_df['ped_no']))
eucledian_for_all = []
displacement_for_all = []
temp_count = 0
temp_count2 = 0
print(len(ped_array))
for j in range(len(ped_array)):
    # if j == 5: break
    test_df_indiv = test_df.loc[test_df['ped_no'] == ped_array[j]]
    
    input_xy = np.array(test_df_indiv[["pos_x", "pos_y"]]).astype('float32')
    # print("pedestrian no: ", ped_array[j], "\n")
    if len(input_xy) < 15: 
        # print("length of input_xy", len(input_xy), "\n")
        # print("skipped")
        temp_count2 += 1
        continue

    temp_neighbours = np.array(test_df_indiv[["d_w1", "d_w2", "d_w3", "d_w4", "d_w5", "d_w6", "d_w7", "d_w8", 
                                              "d_w9", "d_w10", "d_w11", "d_w12", "d_w13", "d_w14", "d_w15", 
                                              "d_w16", "d_w17", "d_w18", "d_w19", "d_w20", "d_w21", "d_w22", 
                                              "d_w23", "d_w24", "d_w25", "d_w26"]]).astype('float32')
    neighbour_test = temp_neighbours[0]

    if len(neighbour_test[np.nonzero(neighbour_test)]) > 9:
        temp_count += 1
        # print("we are here")
        dw_list, nw_x_list, nw_y_list = nearest_neighbour_finder(temp_neighbours)
        # print(dw_list)

        n1 = np.array(test_df_indiv[[dw_list[0], nw_x_list[0], nw_y_list[0]]]).astype('float32')
        n2 = np.array(test_df_indiv[[dw_list[1], nw_x_list[1], nw_y_list[1]]]).astype('float32')
        n3 = np.array(test_df_indiv[[dw_list[2], nw_x_list[2], nw_y_list[2]]]).astype('float32')
        n4 = np.array(test_df_indiv[[dw_list[3], nw_x_list[3], nw_y_list[3]]]).astype('float32')
        n5 = np.array(test_df_indiv[[dw_list[4], nw_x_list[4], nw_y_list[4]]]).astype('float32')
        n6 = np.array(test_df_indiv[[dw_list[5], nw_x_list[5], nw_y_list[5]]]).astype('float32')
        n7 = np.array(test_df_indiv[[dw_list[6], nw_x_list[6], nw_y_list[6]]]).astype('float32')
        n8 = np.array(test_df_indiv[[dw_list[7], nw_x_list[7], nw_y_list[7]]]).astype('float32')
        n9 = np.array(test_df_indiv[[dw_list[8], nw_x_list[8], nw_y_list[8]]]).astype('float32')
    else:
        n1 = np.array(test_df_indiv[["d_w1", "n_w_x1", "n_w_y1"]]).astype('float32')
        n2 = np.array(test_df_indiv[["d_w2", "n_w_x2", "n_w_y2"]]).astype('float32')
        n3 = np.array(test_df_indiv[["d_w3", "n_w_x3", "n_w_y3"]]).astype('float32')
        n4 = np.array(test_df_indiv[["d_w4", "n_w_x4", "n_w_y4"]]).astype('float32')
        n5 = np.array(test_df_indiv[["d_w5", "n_w_x5", "n_w_y5"]]).astype('float32')
        n6 = np.array(test_df_indiv[["d_w6", "n_w_x6", "n_w_y6"]]).astype('float32')
        n7 = np.array(test_df_indiv[["d_w7", "n_w_x7", "n_w_y7"]]).astype('float32')
        n8 = np.array(test_df_indiv[["d_w8", "n_w_x8", "n_w_y8"]]).astype('float32')
        n9 = np.array(test_df_indiv[["d_w9", "n_w_x9", "n_w_y9"]]).astype('float32')

    input_xy = np.array(test_df_indiv[["pos_x", "pos_y"]]).astype('float32')

    n_bound = np.array(test_df_indiv[["d_w_bound", "n_w_bound_x", "n_w_bound_y"]]).astype('float32')
    # n_group = np.array(test_df_indiv[["centroid_dist", "rotation_angle"]]).astype('float32')
    n_group = np.array(test_df_indiv["rotation_angle"]).astype('float32')
    n_centroid = np.array(test_df_indiv[["eta_group_x", "eta_group_y"]]).astype('float32')

    velocity = np.array(test_df_indiv[["vel_x", "vel_y"]]).astype('float32')

    step_size = 12
    delta_t = 0.4
    temp_cycle = 3
    estimated_destination = estimated_dest_list[j]
    # print(estimated_destination)
    # temp_clcle = the cycle from where the prediction actually stats
    # input_xy, velocity, n1, n2, n3, n4, n5, n6, n7, n8, n9, n_bound, n_group

    eucleid, disp = get_predictions(step_size, delta_t, temp_cycle, input_xy, velocity, n1, n2, n3, n4, n5, n6, 
                                    n7, n8, n9, n_bound, n_group, n_centroid, estimated_dest=estimated_destination)
    # print(eucleid, disp)
    
    if not math.isnan(eucleid):
        eucledian_for_all.append(eucleid)
        displacement_for_all.append(disp)

eucledian_for_all = np.array(eucledian_for_all)
displacement_for_all = np.array(displacement_for_all)

print(temp_count2)
print("mean eucledian distance: ", np.mean(eucledian_for_all), "\n")
print("mean final displacement: ", np.mean(displacement_for_all), "\n")
        # print("pedestrian no: ", ped_array[j], "\n")
    # if not math.isnan()
        

360
46
mean eucledian distance:  0.09619106 

mean final displacement:  0.13880712 



In [ ]:
eucledian_distance = get_eucledian(real_trajs_reshaped, predicted_trajs_reshaped)

last_displacement = get_last_disp(real_trajs_reshaped, predicted_trajs_reshaped)
print("eucledian distance: ", eucledian_distance)
print("last displacement: ", last_displacement)

In [ ]:
# mean eucledian distance:  0.61246943 

# mean final displacement:  1.2114174 


In [496]:
# for one cycle only
eucledian_distance = get_eucledian(real_trajs_reshaped[:12], predicted_trajs_reshaped[:12])
last_displacement = get_last_disp(real_trajs_reshaped[:12], predicted_trajs_reshaped[:12])
print("eucledian distance: ", eucledian_distance)
print("last displacement: ", last_displacement)

eucledian distance:  0.98691934
last displacement:  1.2742083
